<a href="https://colab.research.google.com/github/davidgonmar/model-compression-exps/blob/main/UVapprox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import time

# Choose the device: CUDA if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Newton's method for computing the polar factor on the device
def polar_newton(A, max_iter=100, tol=1e-6):
    Q = A.clone()
    for i in range(max_iter):
        Q_inv = torch.inverse(Q)
        Q_next = 0.5 * (Q + Q_inv.T)
        # Check for convergence using the Frobenius norm
        if torch.norm(Q_next - Q) < tol:
            Q = Q_next
            break
        Q = Q_next
    return Q

# Compute the polar factor using full SVD: Q = U * V^T
def polar_via_svd(A):
    U, S, Vh = torch.linalg.svd(A, full_matrices=False)
    return U @ Vh

# Example: comparing on a random 1000x1000 matrix on CUDA
n = 4096
A = torch.randn(n, n, device=device)

# Time the SVD method
if device.type == "cuda":
    torch.cuda.synchronize()
start = time.time()
Q_svd = polar_via_svd(A)
if device.type == "cuda":
    torch.cuda.synchronize()
svd_time = time.time() - start

# Time the Newton iteration method
if device.type == "cuda":
    torch.cuda.synchronize()
start = time.time()
Q_newton = polar_newton(A)
if device.type == "cuda":
    torch.cuda.synchronize()
newton_time = time.time() - start

# Compute the difference between the two methods
error = torch.norm(Q_svd - Q_newton)

print("Matrix size: {}x{}".format(n, n))
print("Time using full SVD: {:.6f} sec".format(svd_time))
print("Time using Newton polar method: {:.6f} sec".format(newton_time))
print("Difference (Frobenius norm) between Q's: {:.6e}".format(error.item()))